# Luminosity with TransformerPayne

In [11]:
from spice.models import IcosphereModel
from spice.spectrum import simulate_observed_flux
from spice.spectrum.spectrum import simulate_monochromatic_luminosity
import astropy.units as u
import jax.numpy as jnp
import numpy as np
from transformer_payne import Blackbody
import matplotlib.pyplot as plt
%matplotlib inline

from jax import config

Let's use TransformerPayne with solar parameters. No high resolution is required, since we are not considering sunspots or other inhomogeneities.

In [12]:
bb = Blackbody()
model = IcosphereModel.construct(1000, 1., 1., bb.to_parameters(), bb.stellar_parameter_names)

/Users/mjablons/Documents/stellar-mesh-integration/src/spice/models/mesh_model.py:245: UserWarning: If override_log_g is True, either parameter_names must include one of [logg,loggs,log_g,log_gs,log g,log gs,surface gravity,surface gravities,surface_gravity,surface_gravities], or log_g_index must be passed for log g to be used in the spectrum emulator.
  warnings.warn(f"If override_log_g is True, either parameter_names must include one of [" + ",".join(


To test the bolometric luminosity value, let's choose a wide range of wavelengths.

The observed flux is calculated for the distance of 10 parsecs by default.

Now let's calculate the monochromatic luminosity.

# Filters

Let's use the ```Filter``` subclasses to check the passband luminosities.

In [21]:
from spice.spectrum.filter import JohnsonCousinsB, JohnsonCousinsI, GaiaG, JohnsonCousinsV

jc_b = JohnsonCousinsB()
jc_i = JohnsonCousinsI()
gaia_g = GaiaG()
jc_v = JohnsonCousinsV()

The passband luminosities can be calculated using the ```AB_passband_luminosity``` function.

In [22]:
from spice.spectrum.spectrum import AB_passband_luminosity
C: float = 299792.458  # km/s

In [23]:
# solar_apparent_mag_B = AB_passband_luminosity(jc_b, vws, observed_flux[:, 0])
# solar_apparent_mag_I = AB_passband_luminosity(jc_i, vws, observed_flux[:, 0])
# solar_apparent_mag_G = AB_passband_luminosity(gaia_g, vws, observed_flux[:, 0])
# solar_johnson_v = AB_passband_luminosity(jc_v, vws, observed_flux[:, 0])

# print("Apparent magnitude of the Sun")
# print("Bessel B:\t", solar_apparent_mag_B, "mag")
# print("Bessel I:\t", solar_apparent_mag_I, "mag")
# print("Gaia G:\t\t", solar_apparent_mag_G, "mag")

# print("Johnson V:\t", solar_johnson_v, "mag")

# Bolometric luminosity of the Sun

The bolometric luminosity of the Sun is the energy that is outputted by the whole Sun. To calculate that, we need to obtain the total flux first.

In [24]:
from spice.spectrum import luminosity, absolute_bol_luminosity

In [42]:
vws_3 = jnp.linspace(1., 100000., 100000)
solar_luminosity_3 = luminosity(bb.flux, model, vws_3)
print("\nSun's bolometric luminosity (100000):", 0.9952351*solar_luminosity_3, "erg/s")
print("Sun's absolute bolometric luminosity (100000):", absolute_bol_luminosity(solar_luminosity_3), "mag")


Sun's bolometric luminosity (100000): 3.8020125e+33 erg/s
Sun's absolute bolometric luminosity (100000): 4.742186 mag


In [33]:
# Calculate total surface area of mesh elements
total_mesh_area = jnp.sum(model.areas) * (u.solRad.to(u.cm)**2)
theoretical_solar_area = 4 * jnp.pi * (u.solRad.to(u.cm)**2)

print("Total mesh surface area:", total_mesh_area, "cm^2")
print("Theoretical solar surface area:", theoretical_solar_area, "cm^2")
print("Ratio of mesh to theoretical area:", total_mesh_area/theoretical_solar_area)


Total mesh surface area: 6.053124e+22 cm^2
Theoretical solar surface area: 6.082104402130212e+22 cm^2
Ratio of mesh to theoretical area: 0.9952351


Let's compare that to the theoretical luminosity:

$L=4\pi R^2 \sigma T^4$

In [43]:
sigma = (5.67*1e-8*u.W/(u.m**2)/(u.K**4)).to(u.erg/(u.cm**2)/(u.s)/(u.K**4))

print("Theoretical luminosity of the Sun:", jnp.sum(model.areas)*(u.solRad.to(u.cm)**2)*sigma*(5772*u.K)**4, "erg/s")

Theoretical luminosity of the Sun: 3.8094993e+33 erg/s


In [44]:
0.9952351*solar_luminosity_3/(jnp.sum(model.areas)*(u.solRad.to(u.cm)**2)*sigma*(5772*u.K)**4)

Array(0.9980347, dtype=float32)